### Imports

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns


Matplotlib is building the font cache; this may take a moment.


#### Import dataset

In [85]:
# Load the CSV file
df = pd.read_csv('data/EVS_WVS_Joint_Csv_v5_0.csv')

# Preview the data
df.head()


/var/folders/s2/syv94kps2t3b18qd88_l60m00000gn/T/ipykernel_27783/721299934.py:3: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/EVS_WVS_Joint_Csv_v5_0.csv')


,study,wave,version,versn_s,doi_gesis,doi_wvsa,studytit,uniqid,intrvwr_id,cntry,...,X028,W003,X052,X035_EVS5,X036E_WVS7,V097EF,W005_EVS5,W006E_WVS7,X047_WVS7,X047E_EVS5
0,1,5,5-0-0 (2024-06-24),5-0-0 (2022-05-16),doi:10.4232/1.14320,doi:10.14281/18241.26,EVS5,8050001,7,8,...,4,4,1,52,-4,9,13,-4,-4,3
1,1,5,5-0-0 (2024-06-24),5-0-0 (2022-05-16),doi:10.4232/1.14320,doi:10.14281/18241.26,EVS5,8050002,1,8,...,3,3,-3,63,-4,9,63,-4,-4,1
2,1,5,5-0-0 (2024-06-24),5-0-0 (2022-05-16),doi:10.4232/1.14320,doi:10.14281/18241.26,EVS5,8050003,10,8,...,4,4,-3,63,-4,9,63,-4,-4,4
3,1,5,5-0-0 (2024-06-24),5-0-0 (2022-05-16),doi:10.4232/1.14320,doi:10.14281/18241.26,EVS5,8050004,6,8,...,5,7,-3,63,-4,9,63,-4,-4,1
4,1,5,5-0-0 (2024-06-24),5-0-0 (2022-05-16),doi:10.4232/1.14320,doi:10.14281/18241.26,EVS5,8050005,1,8,...,7,7,-3,-3,-4,8,93,-4,-4,-1


### Data explorations

#### data cleaning 

Drop columns that aren't relevant and modify others into numerical value 

In [133]:
# print non-numeric columns
def print_non_numeric_columns(df):
    non_numeric_columns = df.select_dtypes(exclude=['number']).columns.tolist()

    for coln in non_numeric_columns:
        print(f"{coln}: {df[coln].unique()}")


# drop columns if not already dropped
def drop_columns(df, columns_to_drop):
    # Drop columns only if they exist in the DataFrame
    existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

    # Drop the existing columns
    df.drop(columns=existing_columns_to_drop, inplace=True)

usa_df = df[df['cntry'] == 840]


# drop following columns, doi, cntry_AN, lnge_iso, version, doi_gesis, doi_wvsa, reg_nuts1, reg_nuts2
columns_to_drop = ['doi_gesis','doi_wvsa', 'cntry_AN', 'lnge_iso', 'version', 'reg_nuts1', 'reg_nuts2', 'X002_02B','V002A_01','V001A_01']
drop_columns(usa_df, columns_to_drop)

# set study title to 1 if it starts with EVS5, else 0 for WVS7
usa_df.loc[:, 'studytit'] = usa_df['studytit'].apply(lambda x: 1 if str(x).startswith('EVS5') else 0)
usa_df['studytit'] = usa_df['studytit'].astype(int)

# set versn_s to 2022 if it starts with 5, else 2024, deduced from the data
usa_df.loc[:, 'versn_s'] = usa_df['versn_s'].apply(lambda x: 2022 if str(x).startswith('5') else 2024)
usa_df['versn_s'] = usa_df['versn_s'].astype(int)

print_non_numeric_columns(usa_df)


/var/folders/s2/syv94kps2t3b18qd88_l60m00000gn/T/ipykernel_27783/1904830023.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=existing_columns_to_drop, inplace=True)
/var/folders/s2/syv94kps2t3b18qd88_l60m00000gn/T/ipykernel_27783/1904830023.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_df['studytit'] = usa_df['studytit'].astype(int)
/var/folders/s2/syv94kps2t3b18qd88_l60m00000gn/T/ipykernel_27783/1904830023.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [138]:
# Count rows with any negative value while handling strings
count_negative_rows = (usa_df.apply(pd.to_numeric, errors='coerce') < 0).any(axis=1).sum()

print(count_negative_rows)

# We gather that every row has at least one negative value, so we will replace them with 0 
# since anything negative is either don't know or refused to answer, NA or missing.

# Replace negative values with 0
usa_df = usa_df.applymap(lambda x: 0 if x < 0 else x)

# define function to check rows for 0 values at a specific column
def check_zero_rows(df, column):
    return df[df[column] == 0].shape[0]

print(check_zero_rows(usa_df, 'A001'))

0
3


/var/folders/s2/syv94kps2t3b18qd88_l60m00000gn/T/ipykernel_27783/2776963355.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  usa_df = usa_df.applymap(lambda x: 0 if x < 0 else x)


PCA 2, cluster 



Color 3 plot i farve om de er left, right 


feature selection